## train the RNN model

In [2]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score

features_df = pd.read_csv('features.csv')

# Extract features and corresponding labels
X = features_df.iloc[:, :-1].values  # features 
y = features_df.iloc[:, -1].values  # labels 

# Initialize the LabelEncoder
label_encoder = LabelEncoder()

# Encode the labels
y = label_encoder.fit_transform(y)

# Convert to PyTorch tensors
X = torch.tensor(X, dtype=torch.float32)
y = torch.tensor(y, dtype=torch.int64)

# Split the dataset into training, validation, and testing sets
X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.4, random_state=42)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42)

# Standardize the features
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_val = scaler.transform(X_val)
X_test = scaler.transform(X_test)

# Define the RNN model
class RNN(nn.Module):
    def __init__(self, input_size, hidden_size, num_classes):
        super(RNN, self).__init__()
        self.hidden_size = hidden_size
        self.rnn = nn.RNN(input_size, hidden_size, batch_first=True)
        self.fc = nn.Linear(hidden_size, num_classes)

    def forward(self, x):
        out, _ = self.rnn(x)
        out = self.fc(out[:, -1, :])
        return out

input_size = X_train.shape[1]
hidden_size = 64 
num_classes = len(np.unique(y_train))

model = RNN(input_size, hidden_size, num_classes)

# Define loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Training loop
num_epochs = 300 

for epoch in range(num_epochs):
    model.train()
    X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
    outputs = model(X_train_tensor.unsqueeze(1))  
    optimizer.zero_grad()
    loss = criterion(outputs.squeeze(), y_train)
    loss.backward()
    optimizer.step()
    
    if (epoch + 1) % 1 == 0:
        print(f'Epoch [{epoch+1}/{num_epochs}], Training Loss: {loss.item():.4f}')

    # Validation
    model.eval()
    X_val_tensor = torch.tensor(X_val, dtype=torch.float32)
    with torch.no_grad():
        val_outputs = model(X_val_tensor.unsqueeze(1))
        val_loss = criterion(val_outputs.squeeze(), y_val)
        _, val_predicted = torch.max(val_outputs, 1)
        val_accuracy = accuracy_score(y_val.numpy(), val_predicted.numpy())

    print(f'Epoch [{epoch+1}/{num_epochs}], Validation Loss: {val_loss.item():.4f}, Validation Accuracy: {val_accuracy:.4f}')

# Evaluate the model on the test set
model.eval()
X_test_tensor = torch.tensor(X_test, dtype=torch.float32)
with torch.no_grad():
    test_outputs = model(X_test_tensor.unsqueeze(1))
    _, test_predicted = torch.max(test_outputs, 1)
    test_accuracy = accuracy_score(y_test.numpy(), test_predicted.numpy())

print("Test Accuracy:", test_accuracy)

Epoch [1/300], Training Loss: 0.6315
Epoch [1/300], Validation Loss: 0.5455, Validation Accuracy: 0.8468
Epoch [2/300], Training Loss: 0.5456
Epoch [2/300], Validation Loss: 0.4712, Validation Accuracy: 0.9031
Epoch [3/300], Training Loss: 0.4707
Epoch [3/300], Validation Loss: 0.4070, Validation Accuracy: 0.9401
Epoch [4/300], Training Loss: 0.4061
Epoch [4/300], Validation Loss: 0.3524, Validation Accuracy: 0.9565
Epoch [5/300], Training Loss: 0.3511
Epoch [5/300], Validation Loss: 0.3062, Validation Accuracy: 0.9686
Epoch [6/300], Training Loss: 0.3048
Epoch [6/300], Validation Loss: 0.2674, Validation Accuracy: 0.9765
Epoch [7/300], Training Loss: 0.2659
Epoch [7/300], Validation Loss: 0.2350, Validation Accuracy: 0.9815
Epoch [8/300], Training Loss: 0.2336
Epoch [8/300], Validation Loss: 0.2080, Validation Accuracy: 0.9800
Epoch [9/300], Training Loss: 0.2066
Epoch [9/300], Validation Loss: 0.1854, Validation Accuracy: 0.9800
Epoch [10/300], Training Loss: 0.1841
Epoch [10/300], V

Epoch [92/300], Validation Loss: 0.0235, Validation Accuracy: 0.9907
Epoch [93/300], Training Loss: 0.0250
Epoch [93/300], Validation Loss: 0.0233, Validation Accuracy: 0.9907
Epoch [94/300], Training Loss: 0.0248
Epoch [94/300], Validation Loss: 0.0231, Validation Accuracy: 0.9907
Epoch [95/300], Training Loss: 0.0246
Epoch [95/300], Validation Loss: 0.0229, Validation Accuracy: 0.9907
Epoch [96/300], Training Loss: 0.0244
Epoch [96/300], Validation Loss: 0.0227, Validation Accuracy: 0.9907
Epoch [97/300], Training Loss: 0.0242
Epoch [97/300], Validation Loss: 0.0224, Validation Accuracy: 0.9907
Epoch [98/300], Training Loss: 0.0241
Epoch [98/300], Validation Loss: 0.0222, Validation Accuracy: 0.9914
Epoch [99/300], Training Loss: 0.0239
Epoch [99/300], Validation Loss: 0.0220, Validation Accuracy: 0.9914
Epoch [100/300], Training Loss: 0.0237
Epoch [100/300], Validation Loss: 0.0218, Validation Accuracy: 0.9914
Epoch [101/300], Training Loss: 0.0235
Epoch [101/300], Validation Loss: 

Epoch [176/300], Training Loss: 0.0146
Epoch [176/300], Validation Loss: 0.0121, Validation Accuracy: 0.9964
Epoch [177/300], Training Loss: 0.0145
Epoch [177/300], Validation Loss: 0.0120, Validation Accuracy: 0.9971
Epoch [178/300], Training Loss: 0.0145
Epoch [178/300], Validation Loss: 0.0120, Validation Accuracy: 0.9971
Epoch [179/300], Training Loss: 0.0144
Epoch [179/300], Validation Loss: 0.0119, Validation Accuracy: 0.9971
Epoch [180/300], Training Loss: 0.0143
Epoch [180/300], Validation Loss: 0.0118, Validation Accuracy: 0.9971
Epoch [181/300], Training Loss: 0.0142
Epoch [181/300], Validation Loss: 0.0117, Validation Accuracy: 0.9971
Epoch [182/300], Training Loss: 0.0142
Epoch [182/300], Validation Loss: 0.0117, Validation Accuracy: 0.9971
Epoch [183/300], Training Loss: 0.0141
Epoch [183/300], Validation Loss: 0.0116, Validation Accuracy: 0.9971
Epoch [184/300], Training Loss: 0.0140
Epoch [184/300], Validation Loss: 0.0115, Validation Accuracy: 0.9971
Epoch [185/300], Tr

Epoch [260/300], Training Loss: 0.0095
Epoch [260/300], Validation Loss: 0.0075, Validation Accuracy: 0.9993
Epoch [261/300], Training Loss: 0.0095
Epoch [261/300], Validation Loss: 0.0075, Validation Accuracy: 0.9993
Epoch [262/300], Training Loss: 0.0094
Epoch [262/300], Validation Loss: 0.0074, Validation Accuracy: 0.9993
Epoch [263/300], Training Loss: 0.0094
Epoch [263/300], Validation Loss: 0.0074, Validation Accuracy: 0.9993
Epoch [264/300], Training Loss: 0.0094
Epoch [264/300], Validation Loss: 0.0074, Validation Accuracy: 0.9993
Epoch [265/300], Training Loss: 0.0093
Epoch [265/300], Validation Loss: 0.0073, Validation Accuracy: 0.9993
Epoch [266/300], Training Loss: 0.0093
Epoch [266/300], Validation Loss: 0.0073, Validation Accuracy: 0.9993
Epoch [267/300], Training Loss: 0.0092
Epoch [267/300], Validation Loss: 0.0073, Validation Accuracy: 0.9993
Epoch [268/300], Training Loss: 0.0092
Epoch [268/300], Validation Loss: 0.0072, Validation Accuracy: 0.9993
Epoch [269/300], Tr

## Test phase 

In [3]:
import os
import librosa
import numpy as np
import pandas as pd
from tqdm import tqdm

def extract_features(file_name):
    try:
        audio, sample_rate = librosa.load(file_name, res_type='kaiser_fast')

        # MFCC (Mel-frequency cepstral coefficients)
        mfccs = librosa.feature.mfcc(y=audio, sr=sample_rate, n_mfcc=40)
        mfccs_processed = np.mean(mfccs.T, axis=0)

        # Chroma feature
        chroma_stft = np.mean(librosa.feature.chroma_stft(y=audio, sr=sample_rate).T, axis=0)

        # Spectral contrast
        spectral_contrast = np.mean(librosa.feature.spectral_contrast(y=audio, sr=sample_rate).T, axis=0)

        # Spectral centroid
        spectral_centroids = np.mean(librosa.feature.spectral_centroid(y=audio, sr=sample_rate).T, axis=0)

        # Zero-crossing rate
        zero_crossing_rate = np.mean(librosa.feature.zero_crossing_rate(y=audio).T, axis=0)

        # Spectral rolloff
        spectral_rolloff = np.mean(librosa.feature.spectral_rolloff(y=audio, sr=sample_rate).T, axis=0)

        # Combine all features into a 1D array
        features = np.hstack([mfccs_processed, chroma_stft, spectral_contrast, spectral_centroids, zero_crossing_rate, spectral_rolloff])

        return features
    except Exception as e:
        print(f"Error encountered while parsing file: {file_name}")
        return None

# Specify the directories containing the .mp3 files
directories = ['something']

# Create an empty DataFrame 
features_df = pd.DataFrame()

for directory in directories:
    print(f"Processing files in {directory} directory")
    for filename in tqdm(os.listdir(directory)):
        if filename.endswith('.wav'):
            file_path = os.path.join(directory, filename)
            try:
                features = extract_features(file_path)
                # Append the features to the DataFrame as a new row
                if features is not None:
                    features_series = pd.Series(features)
                    features_df = pd.concat([features_df, features_series], axis=0)  # Concatenate along rows (axis=0)
            except Exception as e:
                print(f"Error encountered while processing file: {file_path}")
                continue


Processing files in something directory


100%|█████████████████████████████████████████████| 1/1 [00:02<00:00,  2.41s/it]


In [4]:
X_new= features_df.T
X_new.head()

,0,1,2,3,4,5,6,7,8,9,...,52,53,54,55,56,57,58,59,60,61
0,-224.141861,123.548615,-11.934964,35.14872,-2.978472,11.988633,-10.192609,7.214376,-13.9877,1.976113,...,20.868756,15.711451,19.730115,19.38868,19.780333,19.137458,41.547228,1692.69833,0.082245,3374.557483


## load the model

In [5]:

loaded_model = RNN( input_size, hidden_size, num_classes)


loaded_model.load_state_dict(torch.load('rnn_model.pth'))


<All keys matched successfully>

In [6]:
loaded_model.eval()  


X_new = scaler.transform(X_new) 
X_new_tensor = torch.tensor(X_new, dtype=torch.float32)

with torch.no_grad():
    new_outputs = loaded_model(X_new_tensor.unsqueeze(1))
    _, new_predicted = torch.max(new_outputs, 1)




In [7]:
print(new_predicted)

tensor([1])


## Sliding the window by 1 second

In [8]:
import pyaudio
import wave
import time
import os
import pandas as pd  
import torch


def extract_features(file_name):
    try:
        audio, sample_rate = librosa.load(file_name, res_type='kaiser_fast')

        # MFCC (Mel-frequency cepstral coefficients)
        mfccs = librosa.feature.mfcc(y=audio, sr=sample_rate, n_mfcc=40)
        mfccs_processed = np.mean(mfccs.T, axis=0)

        # Chroma feature
        chroma_stft = np.mean(librosa.feature.chroma_stft(y=audio, sr=sample_rate).T, axis=0)

        # Spectral contrast
        spectral_contrast = np.mean(librosa.feature.spectral_contrast(y=audio, sr=sample_rate).T, axis=0)

        # Spectral centroid
        spectral_centroids = np.mean(librosa.feature.spectral_centroid(y=audio, sr=sample_rate).T, axis=0)

        # Zero-crossing rate
        zero_crossing_rate = np.mean(librosa.feature.zero_crossing_rate(y=audio).T, axis=0)

        # Spectral rolloff
        spectral_rolloff = np.mean(librosa.feature.spectral_rolloff(y=audio, sr=sample_rate).T, axis=0)

        # Combine all features into a 1D array
        features = np.hstack([mfccs_processed, chroma_stft, spectral_contrast, spectral_centroids, zero_crossing_rate, spectral_rolloff])

        return features
    except Exception as e:
        print(f"Error encountered while parsing file: {file_name}")
        return None

def record_and_classify_audio():
    audio_duration = 60  # Record audio for 1 minute
    sample_rate = 44100  # 44.1kHz
    ten_sec_frames = int(sample_rate / 1024 * 10)  

    audio_frames = []
    p = pyaudio.PyAudio()

    stream = p.open(format=pyaudio.paInt16, channels=1, rate=sample_rate, input=True, frames_per_buffer=1024)
    
    # Initialize the buffer with 1-minute audio data
    print("Initializing audio buffer with 1-minute audio...")
    for _ in range(0, int(sample_rate / 1024 * audio_duration)):
        data = stream.read(1024)
        audio_frames.append(data)
    print("Initialization complete.")

    while True:
        
        print("Sliding the window by 10 seconds...")
        audio_frames = audio_frames[ten_sec_frames:]
        
        for _ in range(0, ten_sec_frames):
            data = stream.read(1024)
            audio_frames.append(data)

        
        output_audio_file = "recorded_audio.wav"
        wf = wave.open(output_audio_file, 'wb')
        wf.setnchannels(1)
        wf.setsampwidth(p.get_sample_size(pyaudio.paInt16))
        wf.setframerate(sample_rate)
        wf.writeframes(b''.join(audio_frames))
        wf.close()

        # extract features 
        recorded_features = extract_features(output_audio_file)

        if recorded_features is not None:
            recorded_df = pd.DataFrame([recorded_features])
            recorded_df = scaler.transform(recorded_df)
            model.eval()
            recorded_tensor = torch.tensor(recorded_df, dtype=torch.float32)
            with torch.no_grad():
                predicted_output = model(recorded_tensor.unsqueeze(1))
                _, predicted_class = torch.max(predicted_output, 1)

            predicted_class_label = label_encoder.inverse_transform(predicted_class.numpy())
            print("Predicted Class:", predicted_class_label[0])

                

            # Delete the audio file
            os.remove(output_audio_file)
        else:
            print("Error occurred while extracting features from recorded audio.")

        time.sleep(0.1)

# Entry point
if __name__ == "__main__":
    record_and_classify_audio()


Initializing audio buffer with 1-minute audio...


KeyboardInterrupt: 

In [12]:
import pyaudio
import wave
import time
import os
import pandas as pd
import torch
import numpy as np
import librosa
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import LabelEncoder


def extract_features(file_name):
    try:
        audio, sample_rate = librosa.load(file_name, res_type='kaiser_fast')

        # MFCC (Mel-frequency cepstral coefficients)
        mfccs = librosa.feature.mfcc(y=audio, sr=sample_rate, n_mfcc=40)
        mfccs_processed = np.mean(mfccs.T, axis=0)

        # Chroma feature
        chroma_stft = np.mean(librosa.feature.chroma_stft(y=audio, sr=sample_rate).T, axis=0)

        # Spectral contrast
        spectral_contrast = np.mean(librosa.feature.spectral_contrast(y=audio, sr=sample_rate).T, axis=0)

        # Spectral centroid
        spectral_centroids = np.mean(librosa.feature.spectral_centroid(y=audio, sr=sample_rate).T, axis=0)

        # Zero-crossing rate
        zero_crossing_rate = np.mean(librosa.feature.zero_crossing_rate(y=audio).T, axis=0)

        # Spectral rolloff
        spectral_rolloff = np.mean(librosa.feature.spectral_rolloff(y=audio, sr=sample_rate).T, axis=0)

        # Combine all features into a 1D array
        features = np.hstack([mfccs_processed, chroma_stft, spectral_contrast, spectral_centroids, zero_crossing_rate, spectral_rolloff])

        return features
    except Exception as e:
        print(f"Error encountered while parsing file: {file_name}")
        return None


def record_and_classify_audio():
    audio_duration = 60
    sample_rate = 44100
    _frames_ = int(sample_rate / 1024 * 20)

    audio_frames = []
    p = pyaudio.PyAudio()
    stream = p.open(format=pyaudio.paInt16, channels=1, rate=sample_rate, input=True, frames_per_buffer=1024)
    
    print("Initializing audio buffer with 1-minute audio...")
    for _ in range(0, int(sample_rate / 1024 * audio_duration)):
        data = stream.read(1024)
        audio_frames.append(data)
    print("Initialization complete.")
    
    while True:
        start_time = time.time()
        
        print("Sliding the window by 0.1 seconds...")
        audio_frames = audio_frames[_frames_:]
        for _ in range(0, _frames_):
            data = stream.read(1024)
            audio_frames.append(data)

        output_audio_file = "recorded_audio.wav"
        wf = wave.open(output_audio_file, 'wb')
        wf.setnchannels(1)
        wf.setsampwidth(p.get_sample_size(pyaudio.paInt16))
        wf.setframerate(sample_rate)
        wf.writeframes(b''.join(audio_frames))
        wf.close()

        recorded_features = extract_features(output_audio_file)
        if recorded_features is not None:
            recorded_df = pd.DataFrame([recorded_features])
            recorded_df = scaler.transform(recorded_df)
            model.eval()
            recorded_tensor = torch.tensor(recorded_df, dtype=torch.float32)
            with torch.no_grad():
                predicted_output = model(recorded_tensor.unsqueeze(1))
                _, predicted_class = torch.max(predicted_output, 1)
            
            predicted_class_label = label_encoder.inverse_transform(predicted_class.numpy())
            print("Predicted Class:", predicted_class_label[0])

            end_time = time.time()
            print(f"Inference Time: {end_time - start_time} seconds")

            os.remove(output_audio_file)
        else:
            print("Error occurred while extracting features from recorded audio.")
        
        time.sleep(0.1)

if __name__ == "__main__":
    record_and_classify_audio()


Initializing audio buffer with 1-minute audio...
Initialization complete.
Sliding the window by 0.1 seconds...
Predicted Class: 1.0
Inference Time: 0.12594366073608398 seconds
Sliding the window by 0.1 seconds...
Predicted Class: 1.0
Inference Time: 0.06918215751647949 seconds
Sliding the window by 0.1 seconds...
Predicted Class: 1.0
Inference Time: 0.06673216819763184 seconds
Sliding the window by 0.1 seconds...
Predicted Class: 1.0
Inference Time: 0.04869794845581055 seconds
Sliding the window by 0.1 seconds...
Predicted Class: 1.0
Inference Time: 0.045487165451049805 seconds
Sliding the window by 0.1 seconds...
Predicted Class: 1.0
Inference Time: 0.061213016510009766 seconds
Sliding the window by 0.1 seconds...
Predicted Class: 1.0
Inference Time: 0.07230806350708008 seconds
Sliding the window by 0.1 seconds...
Predicted Class: 1.0
Inference Time: 0.08084225654602051 seconds
Sliding the window by 0.1 seconds...
Predicted Class: 1.0
Inference Time: 0.060571908950805664 seconds
Slidi

Sliding the window by 0.1 seconds...
Predicted Class: 1.0
Inference Time: 0.05243873596191406 seconds
Sliding the window by 0.1 seconds...
Predicted Class: 1.0
Inference Time: 0.04568338394165039 seconds
Sliding the window by 0.1 seconds...
Predicted Class: 1.0
Inference Time: 0.04495358467102051 seconds
Sliding the window by 0.1 seconds...
Predicted Class: 1.0
Inference Time: 0.04788780212402344 seconds
Sliding the window by 0.1 seconds...
Predicted Class: 1.0
Inference Time: 0.04174923896789551 seconds
Sliding the window by 0.1 seconds...
Predicted Class: 1.0
Inference Time: 0.047867774963378906 seconds
Sliding the window by 0.1 seconds...
Predicted Class: 1.0
Inference Time: 0.04828667640686035 seconds
Sliding the window by 0.1 seconds...
Predicted Class: 1.0
Inference Time: 0.047640323638916016 seconds
Sliding the window by 0.1 seconds...
Predicted Class: 1.0
Inference Time: 0.04669833183288574 seconds
Sliding the window by 0.1 seconds...
Predicted Class: 1.0
Inference Time: 0.0446

Sliding the window by 0.1 seconds...
Predicted Class: 1.0
Inference Time: 0.0467987060546875 seconds
Sliding the window by 0.1 seconds...
Predicted Class: 1.0
Inference Time: 0.04650282859802246 seconds
Sliding the window by 0.1 seconds...
Predicted Class: 1.0
Inference Time: 0.04706168174743652 seconds
Sliding the window by 0.1 seconds...
Predicted Class: 1.0
Inference Time: 0.04504036903381348 seconds
Sliding the window by 0.1 seconds...
Predicted Class: 1.0
Inference Time: 0.04392600059509277 seconds
Sliding the window by 0.1 seconds...
Predicted Class: 1.0
Inference Time: 0.04394030570983887 seconds
Sliding the window by 0.1 seconds...
Predicted Class: 1.0
Inference Time: 0.045057058334350586 seconds
Sliding the window by 0.1 seconds...
Predicted Class: 1.0
Inference Time: 0.04661369323730469 seconds
Sliding the window by 0.1 seconds...
Predicted Class: 1.0
Inference Time: 0.045795440673828125 seconds
Sliding the window by 0.1 seconds...
Predicted Class: 1.0
Inference Time: 0.10766

/home/waqar/.local/lib/python3.10/site-packages/librosa/util/decorators.py:88: UserWarning: Trying to estimate tuning from empty frequency set.
  return f(*args, **kwargs)
/home/waqar/.local/lib/python3.10/site-packages/librosa/util/decorators.py:88: UserWarning: Trying to estimate tuning from empty frequency set.
  return f(*args, **kwargs)


Predicted Class: 1.0
Inference Time: 0.09926939010620117 seconds
Sliding the window by 0.1 seconds...
Predicted Class: 1.0
Inference Time: 0.05002164840698242 seconds


...

/home/waqar/.local/lib/python3.10/site-packages/librosa/util/decorators.py:88: UserWarning: Trying to estimate tuning from empty frequency set.
  return f(*args, **kwargs)


Sliding the window by 0.1 seconds...
Predicted Class: 1.0
Inference Time: 0.04821205139160156 seconds
Sliding the window by 0.1 seconds...
Predicted Class: 1.0
Inference Time: 0.044888973236083984 seconds
Sliding the window by 0.1 seconds...
Predicted Class: 1.0
Inference Time: 0.04822516441345215 seconds
Sliding the window by 0.1 seconds...
Predicted Class: 1.0
Inference Time: 0.04967188835144043 seconds
Sliding the window by 0.1 seconds...
Predicted Class: 1.0
Inference Time: 0.04745292663574219 seconds
Sliding the window by 0.1 seconds...
Predicted Class: 1.0
Inference Time: 0.047979116439819336 seconds
Sliding the window by 0.1 seconds...
Predicted Class: 1.0
Inference Time: 0.04483604431152344 seconds
Sliding the window by 0.1 seconds...
Predicted Class: 1.0
Inference Time: 0.047864437103271484 seconds
Sliding the window by 0.1 seconds...
Predicted Class: 0.0
Inference Time: 0.04818534851074219 seconds
Sliding the window by 0.1 seconds...
Predicted Class: 0.0
Inference Time: 0.046

Sliding the window by 0.1 seconds...
Predicted Class: 0.0
Inference Time: 0.07676911354064941 seconds
Sliding the window by 0.1 seconds...
Predicted Class: 0.0
Inference Time: 0.0666193962097168 seconds
Sliding the window by 0.1 seconds...
Predicted Class: 0.0
Inference Time: 0.047005653381347656 seconds
Sliding the window by 0.1 seconds...
Predicted Class: 0.0
Inference Time: 0.04598402976989746 seconds
Sliding the window by 0.1 seconds...
Predicted Class: 0.0
Inference Time: 0.04673147201538086 seconds
Sliding the window by 0.1 seconds...
Predicted Class: 0.0
Inference Time: 0.04567146301269531 seconds
Sliding the window by 0.1 seconds...
Predicted Class: 0.0
Inference Time: 0.05403566360473633 seconds
Sliding the window by 0.1 seconds...
Predicted Class: 0.0
Inference Time: 0.049510955810546875 seconds
Sliding the window by 0.1 seconds...
Predicted Class: 0.0
Inference Time: 0.05797743797302246 seconds
Sliding the window by 0.1 seconds...
Predicted Class: 0.0
Inference Time: 0.04585

Predicted Class: 1.0
Inference Time: 0.046888113021850586 seconds
Sliding the window by 0.1 seconds...
Predicted Class: 1.0
Inference Time: 0.0761725902557373 seconds
Sliding the window by 0.1 seconds...
Predicted Class: 1.0
Inference Time: 0.04436826705932617 seconds
Sliding the window by 0.1 seconds...
Predicted Class: 1.0
Inference Time: 0.050330162048339844 seconds
Sliding the window by 0.1 seconds...
Predicted Class: 1.0
Inference Time: 0.04457855224609375 seconds
Sliding the window by 0.1 seconds...
Predicted Class: 1.0
Inference Time: 0.07383203506469727 seconds
Sliding the window by 0.1 seconds...
Predicted Class: 1.0
Inference Time: 0.06516051292419434 seconds
Sliding the window by 0.1 seconds...
Predicted Class: 1.0
Inference Time: 0.053252220153808594 seconds
Sliding the window by 0.1 seconds...
Predicted Class: 0.0
Inference Time: 0.050577640533447266 seconds
Sliding the window by 0.1 seconds...
Predicted Class: 0.0
Inference Time: 0.04691672325134277 seconds
Sliding the wi

KeyboardInterrupt: 

## Sliding the window by 0.1 second


In [ ]:
import pyaudio
import wave
import time
import os
import pandas as pd  
import torch


def extract_features(file_name):
    try:
        audio, sample_rate = librosa.load(file_name, res_type='kaiser_fast')

        # MFCC (Mel-frequency cepstral coefficients)
        mfccs = librosa.feature.mfcc(y=audio, sr=sample_rate, n_mfcc=40)
        mfccs_processed = np.mean(mfccs.T, axis=0)

        # Chroma feature
        chroma_stft = np.mean(librosa.feature.chroma_stft(y=audio, sr=sample_rate).T, axis=0)

        # Spectral contrast
        spectral_contrast = np.mean(librosa.feature.spectral_contrast(y=audio, sr=sample_rate).T, axis=0)

        # Spectral centroid
        spectral_centroids = np.mean(librosa.feature.spectral_centroid(y=audio, sr=sample_rate).T, axis=0)

        # Zero-crossing rate
        zero_crossing_rate = np.mean(librosa.feature.zero_crossing_rate(y=audio).T, axis=0)

        # Spectral rolloff
        spectral_rolloff = np.mean(librosa.feature.spectral_rolloff(y=audio, sr=sample_rate).T, axis=0)

        # Combine all features into a 1D array
        features = np.hstack([mfccs_processed, chroma_stft, spectral_contrast, spectral_centroids, zero_crossing_rate, spectral_rolloff])

        return features
    except Exception as e:
        print(f"Error encountered while parsing file: {file_name}")
        return None

def record_and_classify_audio():
    audio_duration = 60  # Record audio for 1 minute
    sample_rate = 44100  # 44.1kHz
    _frames_ = int(sample_rate / 1024 * 0.1)  

    audio_frames = []
    p = pyaudio.PyAudio()

    stream = p.open(format=pyaudio.paInt16, channels=1, rate=sample_rate, input=True, frames_per_buffer=1024)
    
    # Initialize the buffer with 1-minute audio data
    print("Initializing audio buffer with 1-minute audio...")
    for _ in range(0, int(sample_rate / 1024 * audio_duration)):
        data = stream.read(1024)
        audio_frames.append(data)
    print("Initialization complete.")

    while True:
        
        print("Sliding the window by 0.1 seconds...")
        audio_frames = audio_frames[_frames_:]
        
        for _ in range(0, _frames_):
            data = stream.read(1024)
            audio_frames.append(data)

        
        output_audio_file = "recorded_audio.wav"
        wf = wave.open(output_audio_file, 'wb')
        wf.setnchannels(1)
        wf.setsampwidth(p.get_sample_size(pyaudio.paInt16))
        wf.setframerate(sample_rate)
        wf.writeframes(b''.join(audio_frames))
        wf.close()

        # extract features 
        recorded_features = extract_features(output_audio_file)

        if recorded_features is not None:
            recorded_df = pd.DataFrame([recorded_features])
            recorded_df = scaler.transform(recorded_df)
            model.eval()
            recorded_tensor = torch.tensor(recorded_df, dtype=torch.float32)
            with torch.no_grad():
                predicted_output = model(recorded_tensor.unsqueeze(1))
                _, predicted_class = torch.max(predicted_output, 1)

            predicted_class_label = label_encoder.inverse_transform(predicted_class.numpy())
            print("Predicted Class:", predicted_class_label[0])

            # Delete the audio file
            os.remove(output_audio_file)
        else:
            print("Error occurred while extracting features from recorded audio.")

        time.sleep(0.1)

# Entry point
if __name__ == "__main__":
    record_and_classify_audio()


## slide window by 0.2 seconds

In [ ]:
import pyaudio
import wave
import time
import os
import pandas as pd  
import torch


def extract_features(file_name):
    try:
        audio, sample_rate = librosa.load(file_name, res_type='kaiser_fast')

        # MFCC (Mel-frequency cepstral coefficients)
        mfccs = librosa.feature.mfcc(y=audio, sr=sample_rate, n_mfcc=40)
        mfccs_processed = np.mean(mfccs.T, axis=0)

        # Chroma feature
        chroma_stft = np.mean(librosa.feature.chroma_stft(y=audio, sr=sample_rate).T, axis=0)

        # Spectral contrast
        spectral_contrast = np.mean(librosa.feature.spectral_contrast(y=audio, sr=sample_rate).T, axis=0)

        # Spectral centroid
        spectral_centroids = np.mean(librosa.feature.spectral_centroid(y=audio, sr=sample_rate).T, axis=0)

        # Zero-crossing rate
        zero_crossing_rate = np.mean(librosa.feature.zero_crossing_rate(y=audio).T, axis=0)

        # Spectral rolloff
        spectral_rolloff = np.mean(librosa.feature.spectral_rolloff(y=audio, sr=sample_rate).T, axis=0)

        # Combine all features into a 1D array
        features = np.hstack([mfccs_processed, chroma_stft, spectral_contrast, spectral_centroids, zero_crossing_rate, spectral_rolloff])

        return features
    except Exception as e:
        print(f"Error encountered while parsing file: {file_name}")
        return None

def record_and_classify_audio():
    audio_duration = 60  # Record audio for 1 minute
    sample_rate = 44100  # 44.1kHz
    _frames_ = int(sample_rate / 1024 * 0.2)  

    audio_frames = []
    p = pyaudio.PyAudio()

    stream = p.open(format=pyaudio.paInt16, channels=1, rate=sample_rate, input=True, frames_per_buffer=1024)
    
    # Initialize the buffer with 1-minute audio data
    print("Initializing audio buffer with 1-minute audio...")
    for _ in range(0, int(sample_rate / 1024 * audio_duration)):
        data = stream.read(1024)
        audio_frames.append(data)
    print("Initialization complete.")

    while True:
        
        print("Sliding the window by 0.2 seconds...")
        audio_frames = audio_frames[_frames_:]
        
        for _ in range(0, _frames_):
            data = stream.read(1024)
            audio_frames.append(data)

        
        output_audio_file = "recorded_audio.wav"
        wf = wave.open(output_audio_file, 'wb')
        wf.setnchannels(1)
        wf.setsampwidth(p.get_sample_size(pyaudio.paInt16))
        wf.setframerate(sample_rate)
        wf.writeframes(b''.join(audio_frames))
        wf.close()

        # extract features 
        recorded_features = extract_features(output_audio_file)

        if recorded_features is not None:
            recorded_df = pd.DataFrame([recorded_features])
            recorded_df = scaler.transform(recorded_df)
            model.eval()
            recorded_tensor = torch.tensor(recorded_df, dtype=torch.float32)
            with torch.no_grad():
                predicted_output = model(recorded_tensor.unsqueeze(1))
                _, predicted_class = torch.max(predicted_output, 1)

            predicted_class_label = label_encoder.inverse_transform(predicted_class.numpy())
            print("Predicted Class:", predicted_class_label[0])

            # Delete the audio file
            os.remove(output_audio_file)
        else:
            print("Error occurred while extracting features from recorded audio.")

        time.sleep(0.1)

# Entry point
if __name__ == "__main__":
    record_and_classify_audio()


## slide window by 0.3 seconds

In [ ]:
import pyaudio
import wave
import time
import os
import pandas as pd  
import torch


def extract_features(file_name):
    try:
        audio, sample_rate = librosa.load(file_name, res_type='kaiser_fast')

        # MFCC (Mel-frequency cepstral coefficients)
        mfccs = librosa.feature.mfcc(y=audio, sr=sample_rate, n_mfcc=40)
        mfccs_processed = np.mean(mfccs.T, axis=0)

        # Chroma feature
        chroma_stft = np.mean(librosa.feature.chroma_stft(y=audio, sr=sample_rate).T, axis=0)

        # Spectral contrast
        spectral_contrast = np.mean(librosa.feature.spectral_contrast(y=audio, sr=sample_rate).T, axis=0)

        # Spectral centroid
        spectral_centroids = np.mean(librosa.feature.spectral_centroid(y=audio, sr=sample_rate).T, axis=0)

        # Zero-crossing rate
        zero_crossing_rate = np.mean(librosa.feature.zero_crossing_rate(y=audio).T, axis=0)

        # Spectral rolloff
        spectral_rolloff = np.mean(librosa.feature.spectral_rolloff(y=audio, sr=sample_rate).T, axis=0)

        # Combine all features into a 1D array
        features = np.hstack([mfccs_processed, chroma_stft, spectral_contrast, spectral_centroids, zero_crossing_rate, spectral_rolloff])

        return features
    except Exception as e:
        print(f"Error encountered while parsing file: {file_name}")
        return None

def record_and_classify_audio():
    audio_duration = 60  # Record audio for 1 minute
    sample_rate = 44100  # 44.1kHz
    _frames_ = int(sample_rate / 1024 * 0.3)  

    audio_frames = []
    p = pyaudio.PyAudio()

    stream = p.open(format=pyaudio.paInt16, channels=1, rate=sample_rate, input=True, frames_per_buffer=1024)
    
    # Initialize the buffer with 1-minute audio data
    print("Initializing audio buffer with 1-minute audio...")
    for _ in range(0, int(sample_rate / 1024 * audio_duration)):
        data = stream.read(1024)
        audio_frames.append(data)
    print("Initialization complete.")

    while True:
        
        print("Sliding the window by 0.3 seconds...")
        audio_frames = audio_frames[_frames_:]
        
        for _ in range(0, _frames_):
            data = stream.read(1024)
            audio_frames.append(data)

        
        output_audio_file = "recorded_audio.wav"
        wf = wave.open(output_audio_file, 'wb')
        wf.setnchannels(1)
        wf.setsampwidth(p.get_sample_size(pyaudio.paInt16))
        wf.setframerate(sample_rate)
        wf.writeframes(b''.join(audio_frames))
        wf.close()

        # extract features 
        recorded_features = extract_features(output_audio_file)

        if recorded_features is not None:
            recorded_df = pd.DataFrame([recorded_features])
            recorded_df = scaler.transform(recorded_df)
            model.eval()
            recorded_tensor = torch.tensor(recorded_df, dtype=torch.float32)
            with torch.no_grad():
                predicted_output = model(recorded_tensor.unsqueeze(1))
                _, predicted_class = torch.max(predicted_output, 1)

            predicted_class_label = label_encoder.inverse_transform(predicted_class.numpy())
            print("Predicted Class:", predicted_class_label[0])

            # Delete the audio file
            os.remove(output_audio_file)
        else:
            print("Error occurred while extracting features from recorded audio.")

        time.sleep(0.1)

# Entry point
if __name__ == "__main__":
    record_and_classify_audio()


## slide window by 0.4 seconds

In [ ]:
import pyaudio
import wave
import time
import os
import pandas as pd  
import torch


def extract_features(file_name):
    try:
        audio, sample_rate = librosa.load(file_name, res_type='kaiser_fast')

        # MFCC (Mel-frequency cepstral coefficients)
        mfccs = librosa.feature.mfcc(y=audio, sr=sample_rate, n_mfcc=40)
        mfccs_processed = np.mean(mfccs.T, axis=0)

        # Chroma feature
        chroma_stft = np.mean(librosa.feature.chroma_stft(y=audio, sr=sample_rate).T, axis=0)

        # Spectral contrast
        spectral_contrast = np.mean(librosa.feature.spectral_contrast(y=audio, sr=sample_rate).T, axis=0)

        # Spectral centroid
        spectral_centroids = np.mean(librosa.feature.spectral_centroid(y=audio, sr=sample_rate).T, axis=0)

        # Zero-crossing rate
        zero_crossing_rate = np.mean(librosa.feature.zero_crossing_rate(y=audio).T, axis=0)

        # Spectral rolloff
        spectral_rolloff = np.mean(librosa.feature.spectral_rolloff(y=audio, sr=sample_rate).T, axis=0)

        # Combine all features into a 1D array
        features = np.hstack([mfccs_processed, chroma_stft, spectral_contrast, spectral_centroids, zero_crossing_rate, spectral_rolloff])

        return features
    except Exception as e:
        print(f"Error encountered while parsing file: {file_name}")
        return None

def record_and_classify_audio():
    audio_duration = 60  # Record audio for 1 minute
    sample_rate = 44100  # 44.1kHz
    _frames_ = int(sample_rate / 1024 * 0.4)  

    audio_frames = []
    p = pyaudio.PyAudio()

    stream = p.open(format=pyaudio.paInt16, channels=1, rate=sample_rate, input=True, frames_per_buffer=1024)
    
    # Initialize the buffer with 1-minute audio data
    print("Initializing audio buffer with 1-minute audio...")
    for _ in range(0, int(sample_rate / 1024 * audio_duration)):
        data = stream.read(1024)
        audio_frames.append(data)
    print("Initialization complete.")

    while True:
        
        print("Sliding the window by 0.4 seconds...")
        audio_frames = audio_frames[_frames_:]
        
        for _ in range(0, _frames_):
            data = stream.read(1024)
            audio_frames.append(data)

        
        output_audio_file = "recorded_audio.wav"
        wf = wave.open(output_audio_file, 'wb')
        wf.setnchannels(1)
        wf.setsampwidth(p.get_sample_size(pyaudio.paInt16))
        wf.setframerate(sample_rate)
        wf.writeframes(b''.join(audio_frames))
        wf.close()

        # extract features 
        recorded_features = extract_features(output_audio_file)

        if recorded_features is not None:
            recorded_df = pd.DataFrame([recorded_features])
            recorded_df = scaler.transform(recorded_df)
            model.eval()
            recorded_tensor = torch.tensor(recorded_df, dtype=torch.float32)
            with torch.no_grad():
                predicted_output = model(recorded_tensor.unsqueeze(1))
                _, predicted_class = torch.max(predicted_output, 1)

            predicted_class_label = label_encoder.inverse_transform(predicted_class.numpy())
            print("Predicted Class:", predicted_class_label[0])

            # Delete the audio file
            os.remove(output_audio_file)
        else:
            print("Error occurred while extracting features from recorded audio.")

        time.sleep(0.1)

# Entry point
if __name__ == "__main__":
    record_and_classify_audio()


## slide window by 0.5 seconds

In [ ]:
import pyaudio
import wave
import time
import os
import pandas as pd  
import torch


def extract_features(file_name):
    try:
        audio, sample_rate = librosa.load(file_name, res_type='kaiser_fast')

        # MFCC (Mel-frequency cepstral coefficients)
        mfccs = librosa.feature.mfcc(y=audio, sr=sample_rate, n_mfcc=40)
        mfccs_processed = np.mean(mfccs.T, axis=0)

        # Chroma feature
        chroma_stft = np.mean(librosa.feature.chroma_stft(y=audio, sr=sample_rate).T, axis=0)

        # Spectral contrast
        spectral_contrast = np.mean(librosa.feature.spectral_contrast(y=audio, sr=sample_rate).T, axis=0)

        # Spectral centroid
        spectral_centroids = np.mean(librosa.feature.spectral_centroid(y=audio, sr=sample_rate).T, axis=0)

        # Zero-crossing rate
        zero_crossing_rate = np.mean(librosa.feature.zero_crossing_rate(y=audio).T, axis=0)

        # Spectral rolloff
        spectral_rolloff = np.mean(librosa.feature.spectral_rolloff(y=audio, sr=sample_rate).T, axis=0)

        # Combine all features into a 1D array
        features = np.hstack([mfccs_processed, chroma_stft, spectral_contrast, spectral_centroids, zero_crossing_rate, spectral_rolloff])

        return features
    except Exception as e:
        print(f"Error encountered while parsing file: {file_name}")
        return None

def record_and_classify_audio():
    audio_duration = 60  # Record audio for 1 minute
    sample_rate = 44100  # 44.1kHz
    _frames_ = int(sample_rate / 1024 * 0.5)  

    audio_frames = []
    p = pyaudio.PyAudio()

    stream = p.open(format=pyaudio.paInt16, channels=1, rate=sample_rate, input=True, frames_per_buffer=1024)
    
    # Initialize the buffer with 1-minute audio data
    print("Initializing audio buffer with 1-minute audio...")
    for _ in range(0, int(sample_rate / 1024 * audio_duration)):
        data = stream.read(1024)
        audio_frames.append(data)
    print("Initialization complete.")

    while True:
        
        print("Sliding the window by 0.5 seconds...")
        audio_frames = audio_frames[_frames_:]
        
        for _ in range(0, _frames_):
            data = stream.read(1024)
            audio_frames.append(data)

        
        output_audio_file = "recorded_audio.wav"
        wf = wave.open(output_audio_file, 'wb')
        wf.setnchannels(1)
        wf.setsampwidth(p.get_sample_size(pyaudio.paInt16))
        wf.setframerate(sample_rate)
        wf.writeframes(b''.join(audio_frames))
        wf.close()

        # extract features 
        recorded_features = extract_features(output_audio_file)

        if recorded_features is not None:
            recorded_df = pd.DataFrame([recorded_features])
            recorded_df = scaler.transform(recorded_df)
            model.eval()
            recorded_tensor = torch.tensor(recorded_df, dtype=torch.float32)
            with torch.no_grad():
                predicted_output = model(recorded_tensor.unsqueeze(1))
                _, predicted_class = torch.max(predicted_output, 1)

            predicted_class_label = label_encoder.inverse_transform(predicted_class.numpy())
            print("Predicted Class:", predicted_class_label[0])

            # Delete the audio file
            os.remove(output_audio_file)
        else:
            print("Error occurred while extracting features from recorded audio.")

        time.sleep(0.1)

# Entry point
if __name__ == "__main__":
    record_and_classify_audio()


## slide window by 0.6 seconds

In [ ]:
import pyaudio
import wave
import time
import os
import pandas as pd  
import torch


def extract_features(file_name):
    try:
        audio, sample_rate = librosa.load(file_name, res_type='kaiser_fast')

        # MFCC (Mel-frequency cepstral coefficients)
        mfccs = librosa.feature.mfcc(y=audio, sr=sample_rate, n_mfcc=40)
        mfccs_processed = np.mean(mfccs.T, axis=0)

        # Chroma feature
        chroma_stft = np.mean(librosa.feature.chroma_stft(y=audio, sr=sample_rate).T, axis=0)

        # Spectral contrast
        spectral_contrast = np.mean(librosa.feature.spectral_contrast(y=audio, sr=sample_rate).T, axis=0)

        # Spectral centroid
        spectral_centroids = np.mean(librosa.feature.spectral_centroid(y=audio, sr=sample_rate).T, axis=0)

        # Zero-crossing rate
        zero_crossing_rate = np.mean(librosa.feature.zero_crossing_rate(y=audio).T, axis=0)

        # Spectral rolloff
        spectral_rolloff = np.mean(librosa.feature.spectral_rolloff(y=audio, sr=sample_rate).T, axis=0)

        # Combine all features into a 1D array
        features = np.hstack([mfccs_processed, chroma_stft, spectral_contrast, spectral_centroids, zero_crossing_rate, spectral_rolloff])

        return features
    except Exception as e:
        print(f"Error encountered while parsing file: {file_name}")
        return None

def record_and_classify_audio():
    audio_duration = 60  # Record audio for 1 minute
    sample_rate = 44100  # 44.1kHz
    _frames_ = int(sample_rate / 1024 * 0.6)  

    audio_frames = []
    p = pyaudio.PyAudio()

    stream = p.open(format=pyaudio.paInt16, channels=1, rate=sample_rate, input=True, frames_per_buffer=1024)
    
    # Initialize the buffer with 1-minute audio data
    print("Initializing audio buffer with 1-minute audio...")
    for _ in range(0, int(sample_rate / 1024 * audio_duration)):
        data = stream.read(1024)
        audio_frames.append(data)
    print("Initialization complete.")

    while True:
        
        print("Sliding the window by 0.6 seconds...")
        audio_frames = audio_frames[_frames_:]
        
        for _ in range(0, _frames_):
            data = stream.read(1024)
            audio_frames.append(data)

        
        output_audio_file = "recorded_audio.wav"
        wf = wave.open(output_audio_file, 'wb')
        wf.setnchannels(1)
        wf.setsampwidth(p.get_sample_size(pyaudio.paInt16))
        wf.setframerate(sample_rate)
        wf.writeframes(b''.join(audio_frames))
        wf.close()

        # extract features 
        recorded_features = extract_features(output_audio_file)

        if recorded_features is not None:
            recorded_df = pd.DataFrame([recorded_features])
            recorded_df = scaler.transform(recorded_df)
            model.eval()
            recorded_tensor = torch.tensor(recorded_df, dtype=torch.float32)
            with torch.no_grad():
                predicted_output = model(recorded_tensor.unsqueeze(1))
                _, predicted_class = torch.max(predicted_output, 1)

            predicted_class_label = label_encoder.inverse_transform(predicted_class.numpy())
            print("Predicted Class:", predicted_class_label[0])

            # Delete the audio file
            os.remove(output_audio_file)
        else:
            print("Error occurred while extracting features from recorded audio.")

        time.sleep(0.1)

# Entry point
if __name__ == "__main__":
    record_and_classify_audio()


## Sliding the window by 0.7 second


In [ ]:
import pyaudio
import wave
import time
import os
import pandas as pd  
import torch


def extract_features(file_name):
    try:
        audio, sample_rate = librosa.load(file_name, res_type='kaiser_fast')

        # MFCC (Mel-frequency cepstral coefficients)
        mfccs = librosa.feature.mfcc(y=audio, sr=sample_rate, n_mfcc=40)
        mfccs_processed = np.mean(mfccs.T, axis=0)

        # Chroma feature
        chroma_stft = np.mean(librosa.feature.chroma_stft(y=audio, sr=sample_rate).T, axis=0)

        # Spectral contrast
        spectral_contrast = np.mean(librosa.feature.spectral_contrast(y=audio, sr=sample_rate).T, axis=0)

        # Spectral centroid
        spectral_centroids = np.mean(librosa.feature.spectral_centroid(y=audio, sr=sample_rate).T, axis=0)

        # Zero-crossing rate
        zero_crossing_rate = np.mean(librosa.feature.zero_crossing_rate(y=audio).T, axis=0)

        # Spectral rolloff
        spectral_rolloff = np.mean(librosa.feature.spectral_rolloff(y=audio, sr=sample_rate).T, axis=0)

        # Combine all features into a 1D array
        features = np.hstack([mfccs_processed, chroma_stft, spectral_contrast, spectral_centroids, zero_crossing_rate, spectral_rolloff])

        return features
    except Exception as e:
        print(f"Error encountered while parsing file: {file_name}")
        return None

def record_and_classify_audio():
    audio_duration = 60  # Record audio for 1 minute
    sample_rate = 44100  # 44.1kHz
    _frames_ = int(sample_rate / 1024 * 0.7)  

    audio_frames = []
    p = pyaudio.PyAudio()

    stream = p.open(format=pyaudio.paInt16, channels=1, rate=sample_rate, input=True, frames_per_buffer=1024)
    
    # Initialize the buffer with 1-minute audio data
    print("Initializing audio buffer with 1-minute audio...")
    for _ in range(0, int(sample_rate / 1024 * audio_duration)):
        data = stream.read(1024)
        audio_frames.append(data)
    print("Initialization complete.")

    while True:
        
        print("Sliding the window by 0.7 seconds...")
        audio_frames = audio_frames[_frames_:]
        
        for _ in range(0, _frames_):
            data = stream.read(1024)
            audio_frames.append(data)

        
        output_audio_file = "recorded_audio.wav"
        wf = wave.open(output_audio_file, 'wb')
        wf.setnchannels(1)
        wf.setsampwidth(p.get_sample_size(pyaudio.paInt16))
        wf.setframerate(sample_rate)
        wf.writeframes(b''.join(audio_frames))
        wf.close()

        # extract features 
        recorded_features = extract_features(output_audio_file)

        if recorded_features is not None:
            recorded_df = pd.DataFrame([recorded_features])
            recorded_df = scaler.transform(recorded_df)
            model.eval()
            recorded_tensor = torch.tensor(recorded_df, dtype=torch.float32)
            with torch.no_grad():
                predicted_output = model(recorded_tensor.unsqueeze(1))
                _, predicted_class = torch.max(predicted_output, 1)

            predicted_class_label = label_encoder.inverse_transform(predicted_class.numpy())
            print("Predicted Class:", predicted_class_label[0])

            # Delete the audio file
            os.remove(output_audio_file)
        else:
            print("Error occurred while extracting features from recorded audio.")

        time.sleep(0.1)

# Entry point
if __name__ == "__main__":
    record_and_classify_audio()


## Sliding the window by 0.8 second


In [ ]:
import pyaudio
import wave
import time
import os
import pandas as pd  
import torch


def extract_features(file_name):
    try:
        audio, sample_rate = librosa.load(file_name, res_type='kaiser_fast')

        # MFCC (Mel-frequency cepstral coefficients)
        mfccs = librosa.feature.mfcc(y=audio, sr=sample_rate, n_mfcc=40)
        mfccs_processed = np.mean(mfccs.T, axis=0)

        # Chroma feature
        chroma_stft = np.mean(librosa.feature.chroma_stft(y=audio, sr=sample_rate).T, axis=0)

        # Spectral contrast
        spectral_contrast = np.mean(librosa.feature.spectral_contrast(y=audio, sr=sample_rate).T, axis=0)

        # Spectral centroid
        spectral_centroids = np.mean(librosa.feature.spectral_centroid(y=audio, sr=sample_rate).T, axis=0)

        # Zero-crossing rate
        zero_crossing_rate = np.mean(librosa.feature.zero_crossing_rate(y=audio).T, axis=0)

        # Spectral rolloff
        spectral_rolloff = np.mean(librosa.feature.spectral_rolloff(y=audio, sr=sample_rate).T, axis=0)

        # Combine all features into a 1D array
        features = np.hstack([mfccs_processed, chroma_stft, spectral_contrast, spectral_centroids, zero_crossing_rate, spectral_rolloff])

        return features
    except Exception as e:
        print(f"Error encountered while parsing file: {file_name}")
        return None

def record_and_classify_audio():
    audio_duration = 60  # Record audio for 1 minute
    sample_rate = 44100  # 44.1kHz
    _frames_ = int(sample_rate / 1024 * 0.8)  

    audio_frames = []
    p = pyaudio.PyAudio()

    stream = p.open(format=pyaudio.paInt16, channels=1, rate=sample_rate, input=True, frames_per_buffer=1024)
    
    # Initialize the buffer with 1-minute audio data
    print("Initializing audio buffer with 1-minute audio...")
    for _ in range(0, int(sample_rate / 1024 * audio_duration)):
        data = stream.read(1024)
        audio_frames.append(data)
    print("Initialization complete.")

    while True:
        
        print("Sliding the window by 0.8 seconds...")
        audio_frames = audio_frames[_frames_:]
        
        for _ in range(0, _frames_):
            data = stream.read(1024)
            audio_frames.append(data)

        
        output_audio_file = "recorded_audio.wav"
        wf = wave.open(output_audio_file, 'wb')
        wf.setnchannels(1)
        wf.setsampwidth(p.get_sample_size(pyaudio.paInt16))
        wf.setframerate(sample_rate)
        wf.writeframes(b''.join(audio_frames))
        wf.close()

        # extract features 
        recorded_features = extract_features(output_audio_file)

        if recorded_features is not None:
            recorded_df = pd.DataFrame([recorded_features])
            recorded_df = scaler.transform(recorded_df)
            model.eval()
            recorded_tensor = torch.tensor(recorded_df, dtype=torch.float32)
            with torch.no_grad():
                predicted_output = model(recorded_tensor.unsqueeze(1))
                _, predicted_class = torch.max(predicted_output, 1)

            predicted_class_label = label_encoder.inverse_transform(predicted_class.numpy())
            print("Predicted Class:", predicted_class_label[0])

            # Delete the audio file
            os.remove(output_audio_file)
        else:
            print("Error occurred while extracting features from recorded audio.")

        time.sleep(0.1)

# Entry point
if __name__ == "__main__":
    record_and_classify_audio()


## Sliding the window by 0.9 second


In [ ]:
import pyaudio
import wave
import time
import os
import pandas as pd  
import torch


def extract_features(file_name):
    try:
        audio, sample_rate = librosa.load(file_name, res_type='kaiser_fast')

        # MFCC (Mel-frequency cepstral coefficients)
        mfccs = librosa.feature.mfcc(y=audio, sr=sample_rate, n_mfcc=40)
        mfccs_processed = np.mean(mfccs.T, axis=0)

        # Chroma feature
        chroma_stft = np.mean(librosa.feature.chroma_stft(y=audio, sr=sample_rate).T, axis=0)

        # Spectral contrast
        spectral_contrast = np.mean(librosa.feature.spectral_contrast(y=audio, sr=sample_rate).T, axis=0)

        # Spectral centroid
        spectral_centroids = np.mean(librosa.feature.spectral_centroid(y=audio, sr=sample_rate).T, axis=0)

        # Zero-crossing rate
        zero_crossing_rate = np.mean(librosa.feature.zero_crossing_rate(y=audio).T, axis=0)

        # Spectral rolloff
        spectral_rolloff = np.mean(librosa.feature.spectral_rolloff(y=audio, sr=sample_rate).T, axis=0)

        # Combine all features into a 1D array
        features = np.hstack([mfccs_processed, chroma_stft, spectral_contrast, spectral_centroids, zero_crossing_rate, spectral_rolloff])

        return features
    except Exception as e:
        print(f"Error encountered while parsing file: {file_name}")
        return None

def record_and_classify_audio():
    audio_duration = 60  # Record audio for 1 minute
    sample_rate = 44100  # 44.1kHz
    _frames_ = int(sample_rate / 1024 * 0.9)  

    audio_frames = []
    p = pyaudio.PyAudio()

    stream = p.open(format=pyaudio.paInt16, channels=1, rate=sample_rate, input=True, frames_per_buffer=1024)
    
    # Initialize the buffer with 1-minute audio data
    print("Initializing audio buffer with 1-minute audio...")
    for _ in range(0, int(sample_rate / 1024 * audio_duration)):
        data = stream.read(1024)
        audio_frames.append(data)
    print("Initialization complete.")

    while True:
        
        print("Sliding the window by 0.9 seconds...")
        audio_frames = audio_frames[_frames_:]
        
        for _ in range(0, _frames_):
            data = stream.read(1024)
            audio_frames.append(data)

        
        output_audio_file = "recorded_audio.wav"
        wf = wave.open(output_audio_file, 'wb')
        wf.setnchannels(1)
        wf.setsampwidth(p.get_sample_size(pyaudio.paInt16))
        wf.setframerate(sample_rate)
        wf.writeframes(b''.join(audio_frames))
        wf.close()

        # extract features 
        recorded_features = extract_features(output_audio_file)

        if recorded_features is not None:
            recorded_df = pd.DataFrame([recorded_features])
            recorded_df = scaler.transform(recorded_df)
            model.eval()
            recorded_tensor = torch.tensor(recorded_df, dtype=torch.float32)
            with torch.no_grad():
                predicted_output = model(recorded_tensor.unsqueeze(1))
                _, predicted_class = torch.max(predicted_output, 1)

            predicted_class_label = label_encoder.inverse_transform(predicted_class.numpy())
            print("Predicted Class:", predicted_class_label[0])

            # Delete the audio file
            os.remove(output_audio_file)
        else:
            print("Error occurred while extracting features from recorded audio.")

        time.sleep(0.1)

# Entry point
if __name__ == "__main__":
    record_and_classify_audio()


In [ ]:
import cv2

def test_video_capture():
    cap = cv2.VideoCapture(1)
    if not cap.isOpened():
        print("Error: Couldn't open the camera.")
        return

    while True:
        ret, frame = cap.read()
        if not ret:
            print("Error: Couldn't read a frame.")
            break

        cv2.imshow('Test', frame)
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    cap.release()
    cv2.destroyAllWindows()

test_video_capture()
